In [8]:
# клиент
import socket
import json
import struct
import os
import time
import logging

logging.basicConfig(
    filename='client_log.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

PORT = 5555
FORMAT = 'utf-8'
HOST = 'localhost'

def create_save_path():
    current_time = time.localtime()
    date_dir = time.strftime("%d-%m-%Y", current_time)
    time_str = time.strftime("%H-%M-%S", current_time)
    save_path = f"./{date_dir}/{time_str}.json"
    
    if not os.path.exists(f"./{date_dir}"):
        os.makedirs(f"./{date_dir}")
    
    return save_path

def send_command(client_socket, command):
    command_json = json.dumps(command)
    command_length = struct.pack("I", len(command_json))
    client_socket.sendall(command_length)
    client_socket.sendall(command_json.encode(FORMAT))

def receive_file(client_socket, save_path):
    with open(save_path, 'wb') as f:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            f.write(data)
    logging.info(f"Файл сохранен как {save_path}")

def send_signal(client_socket, pid, signal_type):
    command = {
        "action": "signal",
        "pid": pid,
        "signal_type": signal_type
    }
    send_command(client_socket, command)
    logging.info(f"Отправлен сигнал {signal_type} процессу с PID {pid}")
def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        client_socket.connect((HOST, PORT))
        logging.info(f"Подключено к серверу {HOST}:{PORT}")
        
        while True:
            print("Доступные команды:")
            print("1. Обновить информацию о процессах (update)")
            print("2. Отправить сигнал процессу (signal)")
            print("3. Выход (exit)")

            user_input = input("Введите команду: ").strip().lower()
            
            if user_input == "update":
                command = {"action": "update"}
                send_command(client_socket, command)
                
                save_path = create_save_path()
                receive_file(client_socket, save_path)
                print(f"Данные сохранены в {save_path}")
            
            elif user_input == "signal":
                pid = int(input("Введите PID процесса: "))
                signal_type = input("Введите тип сигнала (terminate/kill): ").strip().lower()
                send_signal(client_socket, pid, signal_type)
            
            elif user_input == "exit":
                print("Завершаем работу клиента...")
                break
            
            else:
                print("Неизвестная команда. Попробуйте снова.")
    
    except Exception as e:
        logging.error(f"Ошибка подключения к серверу: {e}")
        print(f"Ошибка подключения: {e}")
    
    finally:
        logging.info("Соединение с сервером закрыто.")
        client_socket.close()

if __name__ == "__main__":
    start_client()


Доступные команды:
1. Обновить информацию о процессах (update)
2. Отправить сигнал процессу (signal)
3. Выход (exit)


Введите команду:  update


Данные сохранены в ./04-03-2025/11-03-49.json
Доступные команды:
1. Обновить информацию о процессах (update)
2. Отправить сигнал процессу (signal)
3. Выход (exit)


Введите команду:  signal
Введите PID процесса:  12345678
Введите тип сигнала (terminate/kill):  terminate


Доступные команды:
1. Обновить информацию о процессах (update)
2. Отправить сигнал процессу (signal)
3. Выход (exit)


Введите команду:  exit


Завершаем работу клиента...
